In [21]:
import pandas as pd
import numpy as np

In [2]:
bitcoin = pd.read_csv("../data/merged_datasets.csv")

In [6]:
# Dump some columns
bitcoin.drop(inplace=True, columns=["Unnamed: 0", "Adj Close", "gold_close", "gold_label", "sp500_close", "sp500_label", \
                                    "isPartial", "neg", "neu", "pos"])

In [9]:
# Create a daily change percent
bitcoin['Daily_Change_Perc'] = bitcoin["Daily_Change"] / bitcoin["Open"]

In [45]:
# Re-add gold
gold = pd.read_csv("../data/gold_modified.csv")
gold['Daily_Change'] = gold["Close/Last"] - gold["Open"]
gold['Daily_Change_Perc'] = gold["Daily_Change"] / gold["Open"]
gold["Increased"] = np.where(gold["Daily_Change"] > 0, 1, 0)
# Get date in right format for merge
gold["Date"] = pd.to_datetime(gold["Date"]).dt.strftime('%Y-%m-%d')
gold_needed_columns = gold[["Date", "Close/Last", "Open", "Daily_Change", "Daily_Change_Perc", "Increased"]]

In [46]:
# Merge Gold Columns Back in
bitcoin = bitcoin.merge(gold_needed_columns, on="Date", suffixes=("", "_Gold"))

In [53]:
# Re-add sp500
sp500 = pd.read_csv("../data/sp_500.csv")
sp500['Daily_Change'] = sp500["Close/Last"] - sp500["Open"]
sp500['Daily_Change_Perc'] = sp500["Daily_Change"] / sp500["Open"]
sp500["Increased"] = np.where(sp500["Daily_Change"] > 0, 1, 0)
# Get date in right format for merge
sp500["Date"] = pd.to_datetime(sp500["Date"]).dt.strftime('%Y-%m-%d')
sp_needed_columns = sp500[["Date", "Close/Last", "Open", "Daily_Change", "Daily_Change_Perc", "Increased"]]

In [54]:
bitcoin = bitcoin.merge(sp_needed_columns, on="Date", suffixes=("", "_SP500"))

In [57]:
# set y label
bitcoin["label"] = bitcoin["Daily_Change_Ind"].shift(1)

In [68]:
for (columnName, columnData) in bitcoin.iteritems():
    print('Column Name : ', columnName)
    print('Any NAs : ', columnData.isnull().any())

Column Name :  Date
Any NAs :  False
Column Name :  Open
Any NAs :  False
Column Name :  High
Any NAs :  False
Column Name :  Low
Any NAs :  False
Column Name :  Close
Any NAs :  False
Column Name :  Volume
Any NAs :  False
Column Name :  Daily_Change
Any NAs :  False
Column Name :  Daily_Change_Ind
Any NAs :  False
Column Name :  MACD
Any NAs :  False
Column Name :  PROC_3
Any NAs :  False
Column Name :  PROC_5
Any NAs :  False
Column Name :  PROC_10
Any NAs :  False
Column Name :  wpr
Any NAs :  False
Column Name :  sto_os
Any NAs :  False
Column Name :  goog_trend_score
Any NAs :  False
Column Name :  count
Any NAs :  False
Column Name :  compound
Any NAs :  False
Column Name :  retweets_count
Any NAs :  False
Column Name :  likes_count
Any NAs :  False
Column Name :  replies_count
Any NAs :  False
Column Name :  compound_weighted_replies
Any NAs :  True
Column Name :  compound_weighted_likes
Any NAs :  False
Column Name :  compound_weighted_retweets
Any NAs :  False
Column Name :  

In [77]:
bitcoin['compound_weighted_replies'] = bitcoin['compound_weighted_replies'].fillna(0)

In [78]:
bitcoin['compound_weighted_replies'].isnull().sum()

0

In [80]:
print(len(bitcoin))
bitcoin = bitcoin.dropna()
print(len(bitcoin))

1578
1577


In [81]:
# Train - Test Split
from sklearn.model_selection import train_test_split
random_state = 12345

train_set, test_set = train_test_split(bitcoin, test_size=0.2, random_state=random_state)

In [82]:
len(train_set)

1261

In [83]:
len(test_set)

316

In [84]:
train_set.to_csv("../models/bitcoin_train.csv")
test_set.to_csv("../models/bitcoin_test.csv")